In [5]:
from instance_pyfile import *
from support_pyfile import *
import time


In [6]:
import google.generativeai as genai

model = genai.GenerativeModel(model_name='gemini-pro')

config = genai.GenerationConfig(temperature=1.0)


In [7]:
best_cur = 999999999
best_now = 999999990
cur_temp = 1.0

In [8]:
description = 'You are given a list of points with coordinates: {' + list2str() + '}. Your task is to find a trace, with the shortest possible length, that trverses each point exactly once.\n'

in_context = 'Below are some previous traces and their lengths. The traces are arranged in descending order based on their lengths, where lower values are better.\n'
#in_context += pool2examples(Pool)                  ! NOTICE that each generation, Pool change.

task_instruction = 'Please follow the instruction step-by-step to generate new trace:\n'
task_instruction += 'Step 1. Select two traces from the above traces.\n'
task_instruction += 'Step 2. Crossover the two traces chosen in Step 1 and generate a new trace.\n'
task_instruction += 'Step 3. Mutate the trace generated in Step 2 and generate a new trace.\n'
task_instruction += ('Step 4. Keep the generated trace generated in Step 3, repeat Step 1, 2, 3, until you have ' + str(N) + ' generated traces.\n')
task_instruction += 'Directly give me all the chosen traces at Step 1, bracketed them with <selection> and </selection>, and all the generated traces at Step 3, bracketed them with <res> and </res>. Not any explanation needed.'

In [9]:
Pool = randomFirstN(n, N)

g = 1

while g <= G :

    start_gene = time.time()

    prompt = description + (in_context + pool2examples(Pool)) + task_instruction

    listOff = []
    best_cur = Pool[N-1].length

    while len(listOff) < N :

        config = genai.GenerationConfig(temperature=cur_temp)

        start_call = time.time()

        response = model.generate_content(contents=prompt, generation_config=config)

        spent_call = time.time() - start_call

        print(f"Time for the API calls in generation {g}: {spent_call}")

        newGen = cutGenTrace(response.text)
        for s in newGen:
            if checkPermu(s, n) :
                listOff.append(s)
    P_sharp = transform(listOff[:N])

    Pool = updatePool(Pool, P_sharp, N)
    
    best_now = Pool[N-1].length

    if check_stuck(best_cur, best_now, K=10):
        cur_temp += 0.1
        if cur_temp == 2.0 :
            cur_temp = 1.5
            
        print("update_temp!")

    spent_gene = time.time() - start_gene
    print(f"Total time for the generation {g}: {spent_gene}")
    print(f"best solution now is: {best_now}")

    g += 1

print(Pool[N-1])

Time for the API calls in generation 1: 6.29667067527771
Time for the API calls in generation 1: 20.442533493041992
Time for the API calls in generation 1: 29.332602977752686
got stuck 1 time
Total time for the generation 1: 56.0741024017334
best solution now is: 5300.0
Time for the API calls in generation 2: 18.31947350502014
Time for the API calls in generation 2: 18.231491565704346
Time for the API calls in generation 2: 24.726271390914917
got update!
Total time for the generation 2: 61.27793526649475
best solution now is: 5059.0
Time for the API calls in generation 3: 20.387330055236816
Time for the API calls in generation 3: 21.195043087005615
got stuck 1 time
Total time for the generation 3: 41.58337926864624
best solution now is: 5059.0
Time for the API calls in generation 4: 29.314780950546265
Time for the API calls in generation 4: 22.487988710403442
got update!
Total time for the generation 4: 51.804216146469116
best solution now is: 4790.0
Time for the API calls in generatio

IndexError: list assignment index out of range